# Fetch Data Analyst Take Home

In [13]:
import pandas as pd
import numpy as np
import datetime
import janitor
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import config

In [14]:
# Load big query credentials from the service account file
credentials = service_account.Credentials.from_service_account_file(config.config_vars['service_account'])

---
# Part One: Explore data

In this section, we will be cleaning the data and processing it into BigQuery for SQL querying. 

## Cleaning the data

The clean_df function is a data cleaning pipeline that processes a CSV file by standardizing column names, removing empty rows and columns, replacing invalid or missing values (such as blank spaces and 'zero'), and converting specific columns to appropriate formats. It converts date columns to datetime, barcode columns to integers, and transaction-related columns to numeric values. Additionally, it removes duplicate rows to ensure data integrity. 

After applying these transformations, the cleaned DataFrame is returned, making it ready for further analysis or processing. This function helps ensure that the raw data is structured, consistent, and optimized for analysis.

In [15]:
def clean_df(file_path):
    """
    Reads a CSV file and applies common cleaning operations using pyjanitor.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - pd.DataFrame: A cleaned DataFrame.
    """
     
    df = (
        pd.read_csv(file_path)
        .clean_names()        
        .remove_empty()       
        .dropna(how="all")    
    )

    # Convert blanks into nulls and 'zero' into 0
    df.replace({'': np.nan, ' ': np.nan, 'zero': 0}, inplace=True)

    # Convert columns with 'date' in their name to datetime
    for col in df.columns:
        if 'date' in col.lower(): 
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Convert barcode to integer
    for col in df.columns:
        if 'barcode' in col.lower():
            df[col] = df[col].astype('Int64')
            
    # Convert transaction columns to numeric
    for col in df.columns:
        if 'final' in col.lower():
            df[col] = pd.to_numeric(df[col], errors='coerce')

    df.drop_duplicates(inplace=True) 
    
    return df

In [16]:
users = clean_df('USER_TAKEHOME.csv')
prod = clean_df('PRODUCTS_TAKEHOME.csv')
trans = clean_df('TRANSACTION_TAKEHOME.csv')

## Loading into BigQuery

The load_gbq function is designed to upload a Pandas DataFrame into a specified dataset within BigQuery. It uses the pandas_gbq.to_gbq() method to load the data into the cloud, specifying the project ID, dataset, and table name from the configuration file. The if_exists='replace' option ensures that any existing table with the same name is replaced with the new data. After successfully loading the data, a message is printed indicating how many rows and columns were uploaded, providing confirmation of the successful operation.

In [17]:
def load_gbq(df, output_tbl):
    """
    Load a DataFrame to BigQuery.

    Parameters:
    - df (pd.DataFrame): The DataFrame to load.
    - output_tbl (str): The name of the output table in BigQuery.
    """

    # Load the DataFrame to BigQuery
    pandas_gbq.to_gbq(
        df,
        destination_table=f"{config.config_vars['output_dataset']}.{output_tbl}",
        project_id=config.config_vars['project_id'],
        if_exists='replace',
        credentials=credentials
    )

    # Print success message with the correct table name
    print(f'Loaded {df.shape[0]} rows and {df.shape[1]} columns to {output_tbl}')

In [18]:
load_gbq(users,'tbl_users')
load_gbq(prod,'tbl_products')
load_gbq(trans,'tbl_transactions')

Loaded 100000 rows and 6 columns to tbl_users
Loaded 845337 rows and 7 columns to tbl_products
Loaded 49829 rows and 8 columns to tbl_transactions


---
# Part Two: Provide SQL Queries

In this section, we will be answering three questions of interest by querying our recently uploaded tables from BigQuery. Special attention will be given to our open-ended question where a fair amount of analysis was conducted.

## Close-ended questions

In [ ]:
# Initialize BigQuery client with credentials
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [30]:
query_1 = """
-- Retrieves the top 5 brands by total sales among users who have had their account for at least six months.
SELECT 
  brand 
  ,ROUND(SUM(final_sale), 2) AS total_sale 
FROM 
  `fetch-takehome.fetch_data.tbl_transactions` trans 
-- Filters users with accounts older than 6 months.
INNER JOIN `fetch-takehome.fetch_data.tbl_users` users
  ON users.id = trans.user_id
  AND CAST(created_date AS DATE) <= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)  -- Filters users who have had an account for at least 6 months
-- Retrieves product brand information
INNER JOIN `fetch-takehome.fetch_data.tbl_products` prod
  ON prod.barcode = trans.barcode 
  AND brand IS NOT NULL  -- Excludes products with null brands
GROUP BY 
  brand 
HAVING 
  total_sale > 0  -- Only include brands with a total sales greater than 0
ORDER BY 
  total_sale DESC  -- Order the results by total sales in descending order to get the highest sales first
LIMIT 5;  -- Limit the output to the top 5 brands
"""

# Execute the query
query_job = client.query(query_1)

# Print header
print('Top 5 brands by sales among users that have had their account for at least six months')
print()
print(f"{'Brand':<20} {'Total Sales':>10}")
print("-" * 30)

# Loop through the query results and print each row in a table-like format
for row in query_job.result():
    print(f"{row['brand']:<20} {row['total_sale']:>10,.2f}")

Top 5 brands by sales among users that have had their account for at least six months

Brand                Total Sales
------------------------------
CVS                       72.00
TRIDENT                   46.72
DOVE                      42.88
COORS LIGHT               34.96
AXE                       15.98


In [29]:
query_2 = """
-- Calculates total sales and percentage of sales by generation for the 'Health & Wellness' category
WITH 
  gen_sales AS (
    SELECT 
      -- Categorizing users based on birth year into generations
      CASE 
        WHEN EXTRACT(YEAR FROM birth_date) BETWEEN 2013 AND 2025 THEN 'Gen Alpha'  
        WHEN EXTRACT(YEAR FROM birth_date) BETWEEN 1997 AND 2012 THEN 'Gen Z'        
        WHEN EXTRACT(YEAR FROM birth_date) BETWEEN 1981 AND 1996 THEN 'Millennials'  
        WHEN EXTRACT(YEAR FROM birth_date) BETWEEN 1965 AND 1980 THEN 'Gen X'        
        WHEN EXTRACT(YEAR FROM birth_date) BETWEEN 1946 AND 1964 THEN 'Boomers'      
        ELSE 'Other'                                                               
      END AS generation
      -- Calculating total sales for each generation
      ,ROUND(SUM(trans.final_sale),2) AS total_sales
    FROM 
      `fetch-takehome.fetch_data.tbl_transactions` trans
    INNER JOIN `fetch-takehome.fetch_data.tbl_users` users
      ON users.id = trans.user_id
    INNER JOIN `fetch-takehome.fetch_data.tbl_products` prod
      ON prod.barcode = trans.barcode 
      AND prod.category_1 = 'Health & Wellness'
    GROUP BY 
      generation
)
-- Calculating percentage of sales by generation
SELECT 
  generation
  ,total_sales
  -- Percentage of total sales by generation
  ,ROUND(total_sales / SUM(total_sales) OVER (),2) AS percentage_of_sales
FROM 
gen_sales
-- Ordering by the gen hierarchy
ORDER BY 
  CASE 
    WHEN generation = 'Gen Alpha' THEN 1
    WHEN generation = 'Gen Z' THEN 2
    WHEN generation = 'Millennials' THEN 3
    WHEN generation = 'Gen X' THEN 4
    WHEN generation = 'Boomers' THEN 5
    ELSE 6
  END;
"""

# Execute the query
query_job = client.query(query_2)

# Print header
print('Total Sales in the Health & Wellness Category by Generation')
print()
print(f"{'Generation':<12} {'Total Sales':>12} {'Percentage of Sales':>18}")
print("-" * 42)

# Loop through the query results and print each row in a table-like format
for row in query_job.result():
    print(f"{row['generation']:<12} {row['total_sales']:>12,.2f} {row['percentage_of_sales']:>18,.2f}")

Total Sales in the Health & Wellness Category by Generation

Generation    Total Sales Percentage of Sales
------------------------------------------
Millennials         59.13               0.31
Gen X               41.50               0.22
Boomers             89.03               0.47


## Open-ended questions

To determine Fetch’s power users, we will use the RFM (Recency, Frequency, Monetary) methodology, a marketing analysis tool used to identify a company's best customers by measuring and examining how recently a customer has purchased (Recency), how often they purchase (Frequency), how much the customer spends (Monetary). The table below outlines how each metric is defined in the context of Fetch.

Metric | Definition
--- | -----------
Recency | # of days since last receipt scanned
Frequency | # of unique receipts scanned
Monetary | Total sum of revenue from user transactions

Each metric is divided into three buckets based on ntile() and scored from 1 to 3, where 1 indicates the best performance. These scores are then summed to produce a cumulative score (k), which is used to classify customers into segments.

Segmentation | 	Cumulative Score (k)
--- | -----------
High | ≥ 8
Medium | 6 - 7
Low | < 6

Advantages:
- Direct Revenue Insight: RFM analysis, particularly with the monetary component, provides direct insight into how much value users contribute to the app in terms of spending, helping prioritize high-revenue users.
- High-Value User Identification: The RFM model helps identify high-value users who engage frequently or make significant purchases, allowing Fetch to focus on the most active and valuable customers.
- Targeted Marketing: RFM scores enable more effective marketing campaigns, such as targeting high-scoring users with exclusive offers or loyalty programs to retain their engagement.
- Comprehensive Engagement Tracking: By combining Recency (how recently users engaged), Frequency (how often they engage), and Monetary (how much they spend), RFM provides a complete view of user behavior, identifying both frequent and high-value users.

Potential Bias:
- Monetary Bias: Users who purchase fewer, more expensive items may score higher in the monetary category, even if their overall engagement is low, skewing results by prioritizing high spenders over highly engaged users.
- Inconsistent Spending Patterns: Some users may make infrequent but large purchases, placing them in the high monetary category despite lower overall engagement, misrepresenting their loyalty.
- Excluding Non-Revenue Users: Users who engage heavily but do not directly contribute to revenue (e.g., those who use app features but do not purchase) may be undervalued or overlooked.

In [31]:
query_3 = """
-- Produces a K score for each user based on recency, frequency, and monetary metrics. It then determines the number of users who are considered powers users. 
WITH
  -- Calculate Recency, Frequency, and Monetary metrics for each user
  rfm_data AS (
    SELECT
      user_id
      ,MAX(CAST(scan_date AS DATE)) AS scan_date
      ,DATE_DIFF(CAST(MAX(scan_date) AS DATE), CURRENT_DATE(),DAY) AS recency -- Difference in days from last transaction to current date
      ,COUNT(DISTINCT receipt_id) AS frequency -- Number of transactions
      ,ROUND(SUM(final_sale), 2) AS monetary --Total amount spent by user
    FROM 
      `fetch-takehome.fetch_data.tbl_transactions` trans
    GROUP BY 
      user_id
    HAVING 
      monetary > 0 -- only include users with a total sales greater than 0
  ),
  -- Apply NTILE function to divide users into 3 buckets for each RFM metric
  tiles AS (
    SELECT 
      user_id 
      ,NTILE(3) OVER (ORDER BY recency DESC) AS ptile_recency
      ,NTILE(3) OVER (ORDER BY frequency) AS ptile_frequency
      ,NTILE(3) OVER (ORDER BY monetary) AS ptile_monetary
    FROM
      rfm_data
  ), 
  -- Calculate cumulative RFM score and classify users into segments where 1 is high value and 3 is low value
  rfm_buckets AS (
      SELECT *
      ,ptile_recency + ptile_frequency + ptile_monetary AS cumulative_sum
      ,CASE 
        WHEN (ptile_recency + ptile_frequency + ptile_monetary) >= 8 THEN 1
        WHEN (ptile_recency + ptile_frequency + ptile_monetary) < 6 THEN 3
        ELSE 2
        END AS rfm_segment
      FROM
        tiles
  )
-- Aggregate results and calculate percentage of total users per segment
SELECT
  rfm_segment
  ,COUNT(*) AS user_count
  ,SUM(COUNT(*)) OVER() AS total_user_count
  ,ROUND(COUNT(*)/SUM(COUNT(*)) OVER(),2) AS percent_of_total
FROM
  rfm_buckets
GROUP BY 
  rfm_segment
ORDER BY 
  rfm_segment
"""

# Execute the query
query_job = client.query(query_3)

# Print header
print('RFM Segment Analysis')
print()
print(f"{'RFM Segment':<12} {'User Count':>12} {'Total User Count':>18} {'Percent of Total':>18}")
print("-" * 60)

# Loop through the query results and print each row in a table-like format
for row in query_job.result():
    print(f"{row['rfm_segment']:<12} {row['user_count']:>12} {row['total_user_count']:>18} {row['percent_of_total']:>18.2f}")

RFM Segment Analysis

RFM Segment    User Count   Total User Count   Percent of Total
------------------------------------------------------------
1                    2462              17518               0.14
2                    8677              17518               0.50
3                    6379              17518               0.36
